<a href="https://colab.research.google.com/github/cuauhtemocbe/Diplomado-Ciencia-Datos/blob/main/notebooks/16-Manuscrita.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clasificación de letras

In [1]:
# Misc
from warnings import filterwarnings

# Data wrangling
import h5py
import numpy as np
import pandas as pd

# Plotting
import cufflinks as cf

# Modeling
from keras.layers import Dense
from keras import metrics, Input
from sklearn.neural_network import MLPClassifier
from keras.models import Sequential, Model, load_model
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

# Preprocessing
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from tensorflow.keras.utils import to_categorical

# Model performance
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from sklearn.model_selection import train_test_split, cross_val_score

# Environment setup
filterwarnings("ignore")
cf.set_config_file(offline=True, theme="solar")

In [2]:
df = pd.read_csv('letters.csv', index_col=0)
print(df.shape)
df.head()

(124800, 785)


,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,1x10,...,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28,letter
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,W
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,G
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,P
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,O
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,W


In [3]:
df["letter"].value_counts().reset_index().sort_values("letter", ascending=True)

,letter,count
6,A,4800
10,B,4800
3,C,4800
4,D,4800
11,E,4800
25,F,4800
1,G,4800
5,H,4800
8,I,4800
17,J,4800


## Preprocesamiento

In [4]:
from sklearn.preprocessing import OneHotEncoder

X = df[[x for x in df.columns if x!= "letter"]].astype(float).to_numpy()
y = df["letter"].to_numpy()
# Aplana cada imagen de 28x28 a un vector de 784 características
X = X.reshape((X.shape[0], 28*28))

## Train Test Validation

In [5]:
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.4, random_state=42, stratify=y
)

X_test, X_val, y_test, y_val = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)

In [6]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_val = scaler.transform(X_val)

label_encoder = LabelEncoder()

y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)
y_val_encoded = label_encoder.transform(y_val)

y_train_categorical = to_categorical(y_train_encoded)
y_test_categorical = to_categorical(y_test_encoded)
y_val_categorical = to_categorical(y_val_encoded)

## Modelado

In [7]:
EPOCHS = 50

### Scikit-Learn

In [8]:
# Configuración del modelo
mlp = MLPClassifier(
    hidden_layer_sizes=(128, 64), max_iter=EPOCHS, random_state=42, verbose=True)

mlp.fit(X_train, y_train_encoded)

Iteration 1, loss = 0.98679642
Iteration 2, loss = 0.51070599
Iteration 3, loss = 0.40470135
Iteration 4, loss = 0.34704727
Iteration 5, loss = 0.30683335
Iteration 6, loss = 0.27887884
Iteration 7, loss = 0.25466234
Iteration 8, loss = 0.23872147
Iteration 9, loss = 0.21948534
Iteration 10, loss = 0.20705573
Iteration 11, loss = 0.19583654
Iteration 12, loss = 0.18328510
Iteration 13, loss = 0.16965944
Iteration 14, loss = 0.16252866
Iteration 15, loss = 0.15830732
Iteration 16, loss = 0.15369009
Iteration 17, loss = 0.14517038
Iteration 18, loss = 0.13952015
Iteration 19, loss = 0.13507152
Iteration 20, loss = 0.12272160
Iteration 21, loss = 0.11802926
Iteration 22, loss = 0.11890974
Iteration 23, loss = 0.11142209
Iteration 24, loss = 0.10845386
Iteration 25, loss = 0.11375090
Iteration 26, loss = 0.10483119
Iteration 27, loss = 0.10518772
Iteration 28, loss = 0.09896593
Iteration 29, loss = 0.09637113
Iteration 30, loss = 0.09015376
Iteration 31, loss = 0.08857785
Iteration 32, los

MLPClassifier(hidden_layer_sizes=(128, 64), max_iter=50, random_state=42,
              verbose=True)

In [9]:
# Evaluación
train_score = mlp.score(X_train, y_train_encoded)

print(f"Accuracy (Train): {train_score:.2f}")

y_pred = mlp.predict(X_val)
accuracy = accuracy_score(y_val_encoded, y_pred)
print(f"Accuracy (Val): {accuracy:.2f}")

y_pred = mlp.predict(X_test)
accuracy = accuracy_score(y_test_encoded, y_pred)
print(f"Accuracy (Test): {accuracy:.2f}")

Accuracy (Train): 0.98
Accuracy (Val): 0.87
Accuracy (Test): 0.87


## Keras-Personal

In [30]:
model = Sequential()
model.add(Input(shape=(X_train.shape[1],)))
# Capas ocultas
model.add(Dense(128, activation='sigmoid'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='tanh'))
model.add(Dense(32, activation='selu'))
# Capa de salida: Clasificación Multiclase
model.add(Dense(len(np.unique(y_train_encoded)), activation="softmax"))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["accuracy"])

model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_18 (Dense)                     │ (None, 128)                 │         100,480 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_19 (Dense)                     │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_20 (Dense)                     │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_21 (Dense)                     │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_22 (Dense)                     │ (None, 26)                  │             858 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 115,834 (452.48 KB)

 Trainable params: 115,834 (452.48 KB)

 Non-trainable params: 0 (0.00 B)

In [31]:
model.fit(
    X_train, y_train_categorical, epochs=EPOCHS, batch_size=32, verbose=1,
    validation_data=(X_val, y_val_categorical)
)

Epoch 1/50
2340/2340 ━━━━━━━━━━━━━━━━━━━━ 17s 5ms/step - accuracy: 0.6256 - loss: 1.3101 - val_accuracy: 0.8077 - val_loss: 0.6413
Epoch 2/50
2340/2340 ━━━━━━━━━━━━━━━━━━━━ 17s 7ms/step - accuracy: 0.8347 - loss: 0.5320 - val_accuracy: 0.8405 - val_loss: 0.5235
Epoch 3/50
2340/2340 ━━━━━━━━━━━━━━━━━━━━ 19s 6ms/step - accuracy: 0.8654 - loss: 0.4221 - val_accuracy: 0.8491 - val_loss: 0.5106
Epoch 4/50
2340/2340 ━━━━━━━━━━━━━━━━━━━━ 14s 6ms/step - accuracy: 0.8814 - loss: 0.3659 - val_accuracy: 0.8529 - val_loss: 0.4833
Epoch 5/50
2340/2340 ━━━━━━━━━━━━━━━━━━━━ 13s 6ms/step - accuracy: 0.8959 - loss: 0.3169 - val_accuracy: 0.8546 - val_loss: 0.4912
Epoch 6/50
2340/2340 ━━━━━━━━━━━━━━━━━━━━ 20s 5ms/step - accuracy: 0.9037 - loss: 0.2903 - val_accuracy: 0.8586 - val_loss: 0.4791
Epoch 7/50
2340/2340 ━━━━━━━━━━━━━━━━━━━━ 17s 4ms/step - accuracy: 0.9117 - loss: 0.2652 - val_accuracy: 0.8570 - val_loss: 0.4903
Epoch 8/50
2340/2340 ━━━━━━━━━━━━━━━━━━━━ 12s 5ms/step - accuracy: 0.9158 - loss: 0

In [32]:
dc_history = model.history.history
accuracy_df = pd.DataFrame(dc_history)
accuracy_df.tail()

,accuracy,loss,val_accuracy,val_loss
45,0.962433,0.096918,0.852043,0.819237
46,0.963849,0.092657,0.850481,0.844429
47,0.962500,0.098531,0.849760,0.839049
48,0.963395,0.094859,0.849079,0.841558
49,0.964677,0.092155,0.851282,0.849218


In [33]:
y_test_pred = model.predict(X_test)
y_test_pred = np.argmax(y_test_pred, axis=1)
accuracy = accuracy_score(y_test_encoded, y_test_pred)

780/780 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


In [34]:
train_score = accuracy_df.iloc[-1]["accuracy"]
val_score = accuracy_df.iloc[-1]["val_accuracy"]
print(f"Accuracy (Train): {train_score:.2f}")
print(f"Accuracy (Val): {val_score:.2f}")
print(f"Accuracy (Test): {accuracy:.2f}")

Accuracy (Train): 0.96
Accuracy (Val): 0.85
Accuracy (Test): 0.85


## Keras - Gemini

**Prompt:** Quiero una red neuronal secuencial usando Keras para clasificar imagenes de letras escritas a mano. Las letras van de la A a la Z, y cada registros esta compuesto por una matriz de 28 x 28.

In [15]:
model = Sequential()
model.add(Input(shape=(X_train.shape[1],)))
# Capas ocultas
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
# Capa de salida: Clasificación Multiclase
model.add(Dense(len(np.unique(y_train_encoded)), activation="softmax"))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["accuracy"])

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_5 (Dense)                      │ (None, 128)                 │         100,480 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 26)                  │             858 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 111,674 (436.23 KB)

 Trainable params: 111,674 (436.23 KB)

 Non-trainable params: 0 (0.00 B)

In [16]:
model.fit(
    X_train, y_train_categorical, epochs=EPOCHS, batch_size=32, verbose=1,
    validation_data=(X_val, y_val_categorical)
)

Epoch 1/50
2340/2340 ━━━━━━━━━━━━━━━━━━━━ 15s 6ms/step - accuracy: 0.6472 - loss: 1.2200 - val_accuracy: 0.8274 - val_loss: 0.5894
Epoch 2/50
2340/2340 ━━━━━━━━━━━━━━━━━━━━ 17s 4ms/step - accuracy: 0.8517 - loss: 0.4828 - val_accuracy: 0.8427 - val_loss: 0.5294
Epoch 3/50
2340/2340 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.8759 - loss: 0.3837 - val_accuracy: 0.8612 - val_loss: 0.4929
Epoch 4/50
2340/2340 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - accuracy: 0.8917 - loss: 0.3306 - val_accuracy: 0.8492 - val_loss: 0.5280
Epoch 5/50
2340/2340 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.8981 - loss: 0.3091 - val_accuracy: 0.8668 - val_loss: 0.4794
Epoch 6/50
2340/2340 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.9075 - loss: 0.2741 - val_accuracy: 0.8704 - val_loss: 0.4648
Epoch 7/50
2340/2340 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.9159 - loss: 0.2496 - val_accuracy: 0.8640 - val_loss: 0.4910
Epoch 8/50
2340/2340 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.9180 - loss: 0.2

In [17]:
dc_history = model.history.history
accuracy_df = pd.DataFrame(dc_history)
accuracy_df.tail()

,accuracy,loss,val_accuracy,val_loss
45,0.958240,0.112039,0.874559,0.883130
46,0.957545,0.116721,0.872556,0.967840
47,0.955155,0.132453,0.873277,0.888822
48,0.956651,0.129204,0.865946,0.946869
49,0.960684,0.102707,0.876843,0.946836


In [18]:
y_test_pred = model.predict(X_test)
y_test_pred = np.argmax(y_test_pred, axis=1)
accuracy = accuracy_score(y_test_encoded, y_test_pred)

780/780 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step


In [19]:
train_score = accuracy_df.iloc[-1]["accuracy"]
val_score = accuracy_df.iloc[-1]["val_accuracy"]
print(f"Accuracy (Train): {train_score:.2f}")
print(f"Accuracy (Val): {val_score:.2f}")
print(f"Accuracy (Test): {accuracy:.2f}")

Accuracy (Train): 0.96
Accuracy (Val): 0.88
Accuracy (Test): 0.88


## Keras - GPT4o

**Prompt:**
Quiero una red neuronal secuencial usando Keras para clasificar imagenes de letras escritas a mano. Las letras van de la A a la Z, y cada registros esta compuesto por una matriz de 28 x 28.
Mi input no es de 28 x 28. El total de filas en el dataframe es de 124800. Por practidad, cada registro del dataset contiene 784 (28x28) variables de entrada, donde cada variable representa el nivel de intensidad de color en cada píxel, después de convertir la magen cuadrada en un vector.

In [26]:
from tensorflow import keras
from tensorflow.keras import layers

model = Sequential()
# Capas ocultas
model.add(Dense(128, activation='relu', input_shape=(784, )))
model.add(Dense(64, activation='relu'))
# Capa de salida: Clasificación Multiclase
model.add(Dense(len(np.unique(y_train_encoded)), activation="softmax"))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["accuracy"])

model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_15 (Dense)                     │ (None, 128)                 │         100,480 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_16 (Dense)                     │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ (None, 26)                  │           1,690 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 110,426 (431.35 KB)

 Trainable params: 110,426 (431.35 KB)

 Non-trainable params: 0 (0.00 B)

In [27]:
history = model.fit(
    X_train, y_train_categorical, epochs=EPOCHS, batch_size=32, verbose=1,
    validation_data=(X_val, y_val_categorical)
)

Epoch 1/50
2340/2340 ━━━━━━━━━━━━━━━━━━━━ 13s 5ms/step - accuracy: 0.6830 - loss: 1.1038 - val_accuracy: 0.8378 - val_loss: 0.5512
Epoch 2/50
2340/2340 ━━━━━━━━━━━━━━━━━━━━ 19s 8ms/step - accuracy: 0.8560 - loss: 0.4619 - val_accuracy: 0.8571 - val_loss: 0.4950
Epoch 3/50
2340/2340 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.8821 - loss: 0.3679 - val_accuracy: 0.8644 - val_loss: 0.4697
Epoch 4/50
2340/2340 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.8943 - loss: 0.3219 - val_accuracy: 0.8623 - val_loss: 0.4761
Epoch 5/50
2340/2340 ━━━━━━━━━━━━━━━━━━━━ 20s 4ms/step - accuracy: 0.9038 - loss: 0.2882 - val_accuracy: 0.8737 - val_loss: 0.4558
Epoch 6/50
2340/2340 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.9123 - loss: 0.2585 - val_accuracy: 0.8727 - val_loss: 0.5061
Epoch 7/50
2340/2340 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - accuracy: 0.9175 - loss: 0.2433 - val_accuracy: 0.8742 - val_loss: 0.5056
Epoch 8/50
2340/2340 ━━━━━━━━━━━━━━━━━━━━ 17s 4ms/step - accuracy: 0.9230 - loss: 0.

In [28]:
dc_history = history.history
accuracy_df = pd.DataFrame(dc_history)
accuracy_df.tail()

,accuracy,loss,val_accuracy,val_loss
45,0.957439,0.134418,0.866987,1.271750
46,0.961205,0.114706,0.870553,1.358302
47,0.959963,0.119346,0.870833,1.314051
48,0.961846,0.108062,0.871274,1.304980
49,0.960296,0.119668,0.864583,1.290408


In [29]:
y_test_pred = model.predict(X_test)
y_test_pred = np.argmax(y_test_pred, axis=1)
accuracy = accuracy_score(y_test_encoded, y_test_pred)

train_score = accuracy_df.iloc[-1]["accuracy"]
val_score = accuracy_df.iloc[-1]["val_accuracy"]
print(f"Accuracy (Train): {train_score:.2f}")
print(f"Accuracy (Val): {val_score:.2f}")
print(f"Accuracy (Test): {accuracy:.2f}")

780/780 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Accuracy (Train): 0.96
Accuracy (Val): 0.86
Accuracy (Test): 0.87


# Conclusiones

En este proyecto, evaluamos varias arquitecturas de redes neuronales secuenciales para abordar un problema de clasificación. Utilizamos cuatro modelos diferentes, incluyendo una implementación con **MLPClassifier de scikit-learn**, una arquitectura personalizada, una sugerida por el modelo **Gimini**, y una sugerida por el modelo **GPT-4o**. A continuación, se resumen los resultados y las principales observaciones:

## 1. Modelo MLPClassifier de scikit-learn
- **Arquitectura**: Dos capas ocultas (128 y 64 neuronas) con activación *ReLU*, y una capa de salida *softmax*. Usamos estas funciones por que son las que ofrece por defecto.
- **Resultados**:
  - Accuracy (Train): 0.98
  - Accuracy (Val): 0.87
  - Accuracy (Test): 0.87
- **Análisis**: Este modelo mostró un excelente rendimiento en los datos de entrenamiento, pero presentó sobreajuste, ya que su desempeño disminuyó en los conjuntos de validación y test.

## 2. Arquitectura Personalizada
- **Arquitectura**: Cuatro capas ocultas con activaciones combinadas (*sigmoid* en la primera capa de 128 neuronas, y *ReLU* (64 neuronas), tanh (64 neuronas) y selu (32 neuronas). La selección de estas capas fue resultado de prueba y error.
- **Resultados**:
  - Accuracy (Train): 0.96
  - Accuracy (Val): 0.85
  - Accuracy (Test): 0.85
- **Análisis**: Aunque este modelo también logró un buen ajuste en el entrenamiento, presentó un rendimiento similar al MLPClassifier en validación y test, reflejando también un problema de sobreajuste.

## 3. Arquitectura porpuesta por Gimini
- **Arquitectura**: Tres capas ocultas (128, 64 y 32 neuronas) con activación *ReLU* en todas las capas.
- **Resultados**:
  - Accuracy (Train): 0.96
  - Accuracy (Val): 0.88
  - Accuracy (Test): 0.88
- **Análisis**: Este modelo mostró un rendimiento consistente, con resultados ligeramente mejores que los anteriores en los conjuntos de validación y test. Sin embargo, el margen de mejora en comparación con las otras arquitecturas es limitado.

## 4. Arquitectura propuesta por GPT-4o
- **Arquitectura**: Dos capas ocultas con activación *ReLU* (128 y 64 neuronas), lo que resulta en una arquitectura más simple que las anteriores.
- **Resultados**:
  - Accuracy (Train): 0.96
  - Accuracy (Val): 0.86
  - Accuracy (Test): 0.87
- **Análisis**: Este modelo, aunque más sencillo en términos de número de capas, produjo resultados competitivos en comparación con las otras arquitecturas más complejas. Esto sugiere que reducir la complejidad del modelo no necesariamente afecta negativamente el rendimiento.

## Observaciones Generales
- **Función de activación**: La función *ReLU* fue dominante en todas las arquitecturas, excepto en nuestra propuesta personalizada, donde se usó una combinación de *sigmoid*, *ReLU*, *tanh* y *selu*.
- **Número de capas**: Las arquitecturas más simples, como las propuestas por GPT-4o, demostraron ser competitivas en comparación con modelos más complejos. Esto subraya la importancia de evitar la sobre-ingeniería en las capas ocultas.
- **Overfitting**: Todos los modelos mostraron buen rendimiento en los datos de entrenamiento, pero un descenso de desempeño en los conjuntos de validación y test, lo que indica la presencia de sobreajuste.
- **128 neuronas**: Fue una propuesta popular para la primer capa entre las arquitecturas. Parece que es una práctica común. Está relacionada con las arquitecturas de las computadores y GPUs y en pruebas empíricas para dataset medianos a grandes.

## Conclusión Final
Aunque las cuatro arquitecturas presentaron un comportamiento similar, el modelo propuesto por GPT-4o, a pesar de ser más simple, obtuvo resultados comparables a los de arquitecturas más complejas. Esto resalta que, en problemas de clasificación similares, una mayor cantidad de capas ocultas no siempre garantiza un mejor rendimiento, y que la simplicidad del modelo puede ser una estrategia eficaz para evitar el sobreajuste.
